# Tree-Based & Ensemble Regression

Compare a CART-style Decision Tree, Random Forest, and Gradient Boosting on the same scenario.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, warnings
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit

from sklearn.inspection import permutation_importance, PartialDependenceDisplay, partial_dependence
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.base import clone
from numpy.random import RandomState

!wget -q https://raw.githubusercontent.com/Jihun-ust/ust-mail-557/main/Regression_Forecasting/reg_for_utils.py
import reg_for_utils as utils
csv_path = "https://raw.githubusercontent.com/Jihun-ust/ust-mail-557/main/Regression_Forecasting/marketing_daily.csv"
warnings.filterwarnings("ignore")

df = pd.read_csv(csv_path, parse_dates=["date"]).sort_values("date")
train, test = utils.time_train_test_split(df, "date", test_days=90)

features = ["search_spend","social_spend","display_spend","promo","price_index","temp_F","rain","is_weekend"]
target = "revenue"
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]

pre = ColumnTransformer([("num", StandardScaler(), ["search_spend","social_spend","display_spend","price_index","temp_F"]),
                         ("cat", OneHotEncoder(drop="if_binary"), ["promo","rain","is_weekend"])])

models = {
    "tree": Pipeline([("pre", pre), ("est", DecisionTreeRegressor(max_depth=6, random_state=42))]),
    "rf": Pipeline([("pre", pre), ("est", RandomForestRegressor(n_estimators=300, max_depth=6, random_state=42, n_jobs=-1))]),
    "gbr": Pipeline([("pre", pre), ("est", GradientBoostingRegressor(random_state=42))])
}

for name, m in models.items():
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    print(name.UPPER() if hasattr(name, "UPPER") else name.upper(), "RMSE:", utils.rmse(y_test, y_pred), "MAE:", utils.mae(y_test, y_pred))

## Feature importance (Random Forest) — intuition for stakeholders

In [ ]:
rf = models["rf"].named_steps["est"]
feat_names = models["rf"].named_steps["pre"].get_feature_names_out()
importances = rf.feature_importances_
imp = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
imp.head(12)

In [ ]:
tree = models["tree"].named_steps["est"]
feat_names = models["tree"].named_steps["pre"].get_feature_names_out()
importances = tree.feature_importances_
imp = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
imp.head(12)

In [ ]:
gbr = models["gbr"].named_steps["est"]
feat_names = models["gbr"].named_steps["pre"].get_feature_names_out()
importances = gbr.feature_importances_
imp = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
imp.head(12)

### (Optional) Diagnose Examples
- Permutation Importances
- Partial Dependence (PDP) & Individual Conditional Expectation (ICE)
- Residual Diagnostics
- Overfitting Diagnostics

In [ ]:
# Choose which fitted model to diagnose
MODEL_KEY = "rf"  # one of: "tree", "rf", "gbr"
estimator = models[MODEL_KEY]

X_tr, y_tr = X_train, y_train
X_va, y_va = X_test, y_test   # treat test as "validation" for diagnostics

def rmse(y_true, y_pred): 
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def final_est(est):
    return est.steps[-1][1] if isinstance(est, Pipeline) else est

def param_key(name):  # for your pipelines where final step is named "est"
    return f"est__{name}"

def feature_names():  # original columns (preprocessor handles transforms internally)
    return list(X_tr.columns)

print(f"Diagnosing model: {MODEL_KEY}  |  features: {feature_names()}")
print("Validation metrics → RMSE: %.3f | MAE: %.3f | R2: %.3f" % (
    rmse(y_va, estimator.predict(X_va)),
    mean_absolute_error(y_va, estimator.predict(X_va)),
    r2_score(y_va, estimator.predict(X_va)),
))

### Permutation Importances

- **Observation:** Each feature’s importance is plotted with a point (mean importance) and error bars (95% confidence interval from bootstrapping).
- **Interpretation:**
  - A **larger mean importance** means the model’s predictions degrade more when that feature is randomly shuffled — i.e., the feature carries more signal.
  - **Wide error bars** suggest uncertainty: the feature’s importance varies depending on which rows were resampled.
  - **Near-zero importances** (and CIs that cross zero) mean the model isn’t really using that feature for predictive power.
- **Leadership takeaway:** This lets you rank which business drivers (e.g., search vs. social spend, weather, price index) the model relies on most, and whether the signal is robust or fragile.

- **Note**: Permutation importances can take much longer to execute than other diagnostics because it repeatedly retrains on resampled data and shuffles each feature many times, multiplying computation cost across features and bootstraps.
  - FYI: Estimated time of executions
    - Basic runtime: **approx. 1 hour** (Google Colab free tier, typically 2-core vCPU, K80 or T4 GPU)
    - Better runtime: **approx. 6 minutes** (Tested on 10-core CPU, 10-core GPU, 16-core Neural Engine)

In [ ]:
# Permutation importances (with bootstrap 95% CI) — FIXED RNG
SCORING = "neg_root_mean_squared_error"  # RMSE-oriented

def permutation_importance_ci(est, X, y, n_repeats=40, n_boot=200, random_state=42):
    rng = RandomState(random_state)  # <- use RandomState, not default_rng/Generator
    feats = feature_names()
    imps = []

    for _ in range(n_boot):
        idx = rng.randint(0, len(X), size=len(X))  # bootstrap indices
        Xb = X.iloc[idx]
        yb = y.iloc[idx]
        # pass RandomState (or an int) to sklearn
        res = permutation_importance(
            est, Xb, yb,
            n_repeats=n_repeats,
            scoring=SCORING,
            random_state=rng,   # OK: RandomState instance
            n_jobs=-1
        )
        imps.append(res.importances_mean)

    imp = np.vstack(imps)
    df_imp = pd.DataFrame({
        "feature": feats,
        "importance_mean": imp.mean(axis=0),
        "ci_lo": np.percentile(imp, 2.5, axis=0),
        "ci_hi": np.percentile(imp, 97.5, axis=0),
    }).sort_values("importance_mean", ascending=False).reset_index(drop=True)
    return df_imp

perm_df = permutation_importance_ci(estimator, X_va, y_va, n_repeats=40, n_boot=200, random_state=42)
display(perm_df)

# Plot
fig, ax = plt.subplots(figsize=(8, max(3, 0.45*len(perm_df))))
ax.errorbar(perm_df["importance_mean"][::-1], np.arange(len(perm_df))[::-1],
            xerr=[perm_df["importance_mean"][::-1] - perm_df["ci_lo"][::-1],
                  perm_df["ci_hi"][::-1] - perm_df["importance_mean"][::-1]],
            fmt="o", capsize=3)
ax.set_yticks(np.arange(len(perm_df))[::-1])
ax.set_yticklabels(perm_df["feature"][::-1])
ax.set_xlabel("Permutation importance (mean; 95% CI)")
ax.set_title(f"Permutation Importances with 95% CI — {MODEL_KEY.upper()} (Validation)")
plt.tight_layout(); plt.show()

### Partial Dependence (PDP) & Individual Conditional Expectation (ICE)

- **PDP (average curves):**
  - Shows the **average relationship** between a single feature and predicted revenue, holding other features constant.
  - Example: If the curve slopes upward for "search_spend," it means higher spend tends to increase predicted revenue on average.
- **ICE (individual lines):**
  - Each thin line is one data point’s predicted revenue as the feature varies across a grid.
  - Divergence among lines shows **heterogeneity**: the feature’s effect depends on context (interactions with other variables).
- **Slope annotations:** Mark key ranges (e.g., 5k→20k spend, 30→80°F) and calculate the model-implied marginal effect. Steeper slopes = larger returns for changes in that range.
- **Leadership takeaway:** PDP/ICE curves help translate model mechanics into **business intuition** (e.g., where marketing dollars are most effective, or when weather starts to affect demand).

In [ ]:
# PDP & ICE for top features (manual, version‑proof) + slope tags

import numpy as np, pandas as pd, matplotlib.pyplot as plt, warnings
from numpy.random import RandomState

TOP_K = 4  # change as desired
rng = RandomState(42)

# Use the permutation importance ranking from Cell 1
top_feats = list(perm_df["feature"][:TOP_K])
print("Top features for PDP/ICE:", top_feats)

# Optional slope annotation ranges—edit if these columns exist in your data
SLOPE_MARKS = {
    "search_spend": [(5_000, 20_000)],
    "social_spend": [(5_000, 20_000)],
    "display_spend": [(5_000, 20_000)],
    "temp_F": [(30, 80)],
}

def get_feature_grid(X, feat, n=50):
    """Make a sensible grid for PDP depending on feature type."""
    x = X[feat].dropna()
    uniq = np.unique(x)
    # Treat small‑cardinality features (incl. binaries) as categorical grid
    if len(uniq) <= 5:
        return np.sort(uniq.astype(float))
    # Robust numeric grid between 1st and 99th percentile
    lo, hi = np.percentile(x, [1, 99])
    if lo == hi:
        lo, hi = x.min(), x.max()
    if lo == hi:
        return np.array([float(lo)])
    return np.linspace(lo, hi, n)

def pdp_manual(est, X, feat, grid):
    """
    Classic PDP: for each grid value v, set X[feat]=v for all rows,
    predict, and average the predictions.
    Works with Pipelines that include preprocessing.
    """
    avgs = []
    for v in grid:
        Xv = X.copy()
        Xv[feat] = v
        yhat = est.predict(Xv)
        avgs.append(np.mean(yhat))
    return np.asarray(avgs)

def ice_manual(est, X_sample, feat, grid):
    """
    ICE: for each grid value v, set X_sample[feat]=v and predict per row.
    Returns matrix shape (n_rows, len(grid)).
    """
    lines = []
    for v in grid:
        Xv = X_sample.copy()
        Xv[feat] = v
        yh = est.predict(Xv)
        lines.append(yh)
    M = np.column_stack(lines)  # n_rows × len(grid)
    return M

def annotate_slope(ax, grid, avg, x0, x1, label):
    i0 = int(np.argmin(np.abs(grid - x0)))
    i1 = int(np.argmin(np.abs(grid - x1)))
    dy = float(avg[i1] - avg[i0])
    dx = float(grid[i1] - grid[i0]) if grid[i1] != grid[i0] else np.nan
    slope = dy/dx if (dx and not np.isnan(dx)) else np.nan
    ax.plot([grid[i0], grid[i1]], [avg[i0], avg[i1]], lw=2)
    ax.scatter([grid[i0], grid[i1]], [avg[i0], avg[i1]], s=35)
    ax.annotate(f"{label}\nΔy={dy:.2f}, Δx={dx:.2f}\nslope={slope:.4f}",
                xy=(grid[i1], avg[i1]), xytext=(6,6), textcoords="offset points",
                bbox=dict(boxstyle="round,pad=0.3", fc="w", ec="0.5", alpha=0.9))

for feat in top_feats:
    # Build grid
    grid = get_feature_grid(X_tr, feat, n=60)

    # PDP (average)
    avg = pdp_manual(estimator, X_tr, feat, grid)
    fig, ax = plt.subplots(figsize=(7,5))
    ax.plot(grid, avg, linewidth=2)
    ax.set_xlabel(feat); ax.set_ylabel("Predicted revenue")
    ax.set_title(f"PDP (average) — {feat}  [{MODEL_KEY.upper()}]")
    plt.tight_layout(); plt.show()

    # ICE (on a subsample to keep it readable)
    n_lines = 200 if len(X_tr) > 200 else len(X_tr)
    rows = rng.choice(X_tr.index.values, size=n_lines, replace=False)
    X_sample = X_tr.loc[rows].copy()
    M = ice_manual(estimator, X_sample, feat, grid)  # shape: n_lines × len(grid)

    fig, ax = plt.subplots(figsize=(7,5))
    for i in range(M.shape[0]):
        ax.plot(grid, M[i, :], alpha=0.06)
    ax.set_xlabel(feat); ax.set_ylabel("Predicted revenue")
    ax.set_title(f"ICE — {feat}  [{MODEL_KEY.upper()}]")
    plt.tight_layout(); plt.show()

    # Optional slope annotations on PDP curve at key ranges
    ranges = SLOPE_MARKS.get(feat, [])
    if len(grid) >= 2 and len(ranges) > 0:
        fig, ax = plt.subplots(figsize=(7,5))
        ax.plot(grid, avg, linewidth=2)
        ax.set_xlabel(feat); ax.set_ylabel("Predicted revenue")
        ax.set_title(f"PDP with slope annotations — {feat}")
        for (x0, x1) in ranges:
            annotate_slope(ax, grid, avg, x0, x1, f"{x0}→{x1}")
        plt.tight_layout(); plt.show()

### Residual Diagnostics

- **Residuals vs. Predicted:** 
  - Scatter plot of prediction error (residual = actual – predicted).
  - Ideally, residuals should be centered around zero, with no obvious pattern.
  - Patterns (e.g., funnel shape or systematic bias at high predictions) suggest the model misfits certain ranges.
- **By Segment (promo / rain / weekend):**
  - Boxplots show whether errors are systematically higher or lower in certain categories.
  - If residuals are consistently positive under "promo=1," the model underestimates promo effects.
- **Over Time:**
  - Rolling average of residuals highlights whether accuracy drifts across seasons or months.
  - Seasonal boxplots (winter, spring, summer, fall) reveal calendar effects the model may not fully capture.
- **Leadership takeaway:** Residual diagnostics reveal **blind spots** — e.g., underpredicting in promotions, overpredicting in rainy weeks, or drift over seasons — which can guide feature engineering or business adjustments.

In [ ]:
# Build a validation-side frame aligned with X_va rows for segmentation
va_df = df.loc[X_va.index, ["date","promo","rain","is_weekend"]].copy()
va_df["y_true"] = y_va.values
va_df["y_pred"] = estimator.predict(X_va)
va_df["resid"]  = va_df["y_true"] - va_df["y_pred"]

# Residuals vs. predicted
fig, ax = plt.subplots(figsize=(6,5))
ax.scatter(va_df["y_pred"], va_df["resid"], alpha=0.6)
ax.axhline(0, linewidth=1, color="k")
ax.set_xlabel("Predicted"); ax.set_ylabel("Residual (y - ŷ)")
ax.set_title(f"Residuals vs Predicted — {MODEL_KEY.upper()} (Validation)")
plt.tight_layout(); plt.show()

# Categorical segments
for seg in ["promo","rain","is_weekend"]:
    fig, ax = plt.subplots(figsize=(6,4))
    groups = [va_df.loc[va_df[seg]==v, "resid"] for v in sorted(va_df[seg].unique())]
    ax.boxplot(groups, labels=[str(v) for v in sorted(va_df[seg].unique())])
    ax.axhline(0, color="k", lw=1)
    ax.set_xlabel(seg); ax.set_ylabel("Residual")
    ax.set_title(f"Residuals by {seg} — {MODEL_KEY.upper()} (Validation)")
    plt.tight_layout(); plt.show()

# Over time (rolling mean to spot drift or seasonality)
ts = va_df.dropna(subset=["date"]).sort_values("date").copy()
ts["roll_mean_resid"] = ts["resid"].rolling(window=8, min_periods=1).mean()  # ~2 months if weekly
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(ts["date"], ts["resid"], alpha=0.25, label="residual")
ax.plot(ts["date"], ts["roll_mean_resid"], lw=2, label="rolling mean (w=8)")
ax.axhline(0, color="k", lw=1)
ax.set_title(f"Residuals Over Time — {MODEL_KEY.upper()} (Validation)")
ax.set_xlabel("date"); ax.set_ylabel("residual"); ax.legend()
plt.tight_layout(); plt.show()

### Overfitting Diagnostics

- **Train vs. Validation RMSE (grid over depth & trees):**
  - **Training RMSE** (dashed lines) shows how well the model fits the training data.
  - **Validation RMSE** (solid lines) shows generalization to unseen data.
  - When train error drops but validation error rises, that’s **overfitting**.
  - Shallow trees (low depth) may **underfit** (both errors high).
  - Deeper trees or too many estimators may overfit.
- **Early-Stopping Curves (Gradient Boosting only):**
  - RMSE plotted against number of boosting iterations.
  - Training error declines steadily; validation error typically drops then rises.
  - The **best iteration** is where validation RMSE is lowest, use this as a natural stopping point to avoid overfitting.
- **Leadership takeaway:** These curves show whether the model is too simple, too complex, or just right, and help set guardrails (e.g., limiting depth or enabling early-stopping) for reliable performance.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor 

# Grid: n_estimators × max_depth (works for RF & GBR; DT uses max_depth only)
def grid_eval(est, Xtr, ytr, Xva, yva,
              n_estimators_list=(50, 100, 200, 400),
              max_depth_list=(None, 3, 5, 7)):
    rows = []
    fe = final_est(estimator)
    is_rf  = isinstance(fe, RandomForestRegressor)
    is_gbr = isinstance(fe, GradientBoostingRegressor)
    is_dt  = isinstance(fe, DecisionTreeRegressor)

    for d in max_depth_list:
        for n in n_estimators_list if (is_rf or is_gbr) else [None]:
            mdl = clone(estimator)
            params = {}
            if d is not None:
                params[param_key("max_depth")] = d
            if n is not None and (is_rf or is_gbr):
                params[param_key("n_estimators")] = n
            if params:
                mdl.set_params(**params)

            mdl.fit(Xtr, ytr)
            ytr_hat = mdl.predict(Xtr)
            yva_hat = mdl.predict(Xva)
            rows.append({
                "model": type(final_est(mdl)).__name__,
                "max_depth": d,
                "n_estimators": n if n is not None else (mdl.get_params().get(param_key("n_estimators")) if (is_rf or is_gbr) else None),
                "RMSE_train": rmse(ytr, ytr_hat),
                "RMSE_val": rmse(yva, yva_hat),
                "R2_val": r2_score(yva, yva_hat),
            })
    return pd.DataFrame(rows)

grid_df = grid_eval(estimator, X_tr, y_tr, X_va, y_va)
display(grid_df.sort_values("RMSE_val").head(10))

# Plot curves vs n_estimators for each depth (only RF/GBR)
fe = final_est(estimator)
if isinstance(fe, (RandomForestRegressor, GradientBoostingRegressor)):
    fig, ax = plt.subplots(figsize=(9,5))
    for d, g in grid_df.groupby("max_depth"):
        g = g.dropna(subset=["n_estimators"]).sort_values("n_estimators")
        if g.empty: 
            continue
        ax.plot(g["n_estimators"], g["RMSE_val"], marker="o", label=f"val depth={d}")
        ax.plot(g["n_estimators"], g["RMSE_train"], linestyle="--", marker="x", alpha=0.6, label=f"train depth={d}")
    ax.set_xlabel("n_estimators"); ax.set_ylabel("RMSE")
    ax.set_title(f"Train vs. Val RMSE across trees & depths — {MODEL_KEY.upper()}")
    ax.legend(ncol=2); plt.tight_layout(); plt.show()

# Early‑stopping style curves (only for GBR)
if isinstance(fe, GradientBoostingRegressor):
    # Refit a cloned model with more estimators to get smooth staged curves
    gbr = clone(estimator).set_params(**{param_key("n_estimators"): 600})
    gbr.fit(X_tr, y_tr)

    # staged_predict runs on the estimator itself (pipeline OK in sklearn≥1.0)
    tr_curve = []
    va_curve = []
    # Extract the final estimator inside the pipeline to iterate staged_predict robustly
    # while feeding transformed data:
    preproc = gbr.named_steps["pre"]
    est = gbr.named_steps["est"]
    Xtr_t = preproc.transform(X_tr)
    Xva_t = preproc.transform(X_va)

    for yhat in est.staged_predict(Xtr_t):
        tr_curve.append(rmse(y_tr, yhat))
    for yhat in est.staged_predict(Xva_t):
        va_curve.append(rmse(y_va, yhat))
    iters = np.arange(1, len(tr_curve)+1)
    best_iter = int(np.argmin(va_curve) + 1)
    print(f"GBR early‑stopping proxy → best_iter={best_iter}, val RMSE={va_curve[best_iter-1]:.3f}")

    fig, ax = plt.subplots(figsize=(9,5))
    ax.plot(iters, tr_curve, label="Train RMSE")
    ax.plot(iters, va_curve, label="Validation RMSE")
    ax.axvline(best_iter, color="k", linestyle="--", label=f"best={best_iter}")
    ax.set_xlabel("Boosting iterations"); ax.set_ylabel("RMSE")
    ax.set_title("Gradient Boosting — Early‑Stopping Curves")
    ax.legend(); plt.tight_layout(); plt.show()
else:
    warnings.warn("Early‑stopping curves skipped (final estimator is not GradientBoostingRegressor).")